This notebook will be used to upload, search and plot using the HUGS Cloud platform

In [ ]:
import os
import sys
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from HUGS.Processing import search
from HUGS.Client import Process
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Axis, Lines, Figure
from random import randint    
from numpy import random as np_random
import numpy as np

## Login

Login to the platform

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
user = User(username="gareth", identity_url=F"{base_url}/identity")
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/19.ac.4c.8b
(please check that this page displays the message 'small mice run noisily')


Check we've logged in successfully

In [ ]:
user.wait_for_login()

True

## Upload

First we want to upload the data to the cloud platform

In [ ]:
# Helper function for test file paths
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
filepath = get_CRDS_path(filename)
processing = Process(service_url=base_url)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")

NameError: name 'Process' is not defined